In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130590

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,141390.6845
2018-02-28,134485.3690
2018-03-31,133044.5931
2018-04-30,127430.7174
2018-05-31,109233.8618
2018-06-30,125105.2280
2018-07-31,137620.0142
2018-08-31,167992.0149
2018-09-30,159848.5414


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    141390.6845
 2018-02-28    134485.3690
 2018-03-31    133044.5931
 2018-04-30    127430.7174
 2018-05-31    109233.8618
 2018-06-30    125105.2280
 2018-07-31    137620.0142
 2018-08-31    167992.0149
 2018-09-30    159848.5414
 2018-10-31    136267.1582
 2018-11-30    155239.2289
 2018-12-31    138466.6988
 2019-01-31    199537.2905
 2019-02-28    154880.9106
 2019-03-31    155737.7615
 2019-04-30    158858.9819
 2019-05-31    141339.4175
 2019-06-30    147280.5702
 2019-07-31    135109.8506
 2019-08-31    167554.2008
 2019-09-30    122931.9809
 2019-10-31    148764.6440
 2019-11-30    130562.3251
 2019-12-31    123500.4132
 2020-01-31    117979.8327
 2020-02-29    122278.4237
 2020-03-31    132518.8186
 2020-04-30    104679.5229
 2020-05-31     94323.7022
 2020-06-30     92185.0717
 2020-07-31    142645.4667
 2020-08-31    133315.7890
 2020-09-30    114676.6628
 2020-10-31    135777.0610
 2020-11-30    125162.3331
 2020-12-31    118850.9088

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.358544
p-value : 0.153780
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -10.178058
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=957.186, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=954.331, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=951.598, Time=0.04 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=954.503, Time=0.05 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=952.677, Time=0.07 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.50 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=949.549, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=949.863, Time=0.02 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=950.910, Time=0.05 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=953.208, Time=0.06 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=953.019, Time=0.13 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=948.572, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=950.690, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=952.418, Time=0.15 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.24 sec
 ARIMA(3,0,0)(0,0,0)[1] intercept

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -470.256
Method:                       css-mle   S.D. of innovations          17559.199
Date:                Sat, 11 Sep 2021   AIC                            948.511
Time:                        04:45:09   BIC                            955.462
Sample:                             0   HQIC                           951.059
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.352e+05   6301.970     21.449      0.000    1.23e+05    1.48e+05
ar.L1.y        0.3295      0.147      2.244      0.025       0.042       0.617
ar.L2.y        0.2596      0.146      1.777      0.0

# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([140867.29809561, 141024.8916573 ]), array([17559.19908263, 18488.03371814]), array([[106451.90029629, 175282.69589493],
       [104789.01142478, 177260.77188982]]))
